### Imports

In [2]:
import cv2
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

### Constants

In [7]:
query_dir = "./data/qsd1_w3/"
bbdd_dir = "./data/BBDD/"

### Helper functions

In [5]:
# images.py
def load_images_from_directory(directory):
    images = []
    # List all files in the directory
    for filename in os.listdir(directory):
        # Filter only image files based on file extensions
        if filename.endswith(('.jpg')):
            # Full path to the image file
            img_path = os.path.join(directory, filename)
            # Read the image using OpenCV
            img = cv2.imread(img_path)
            # Check if the image was loaded successfully
            if img is not None:
                images.append(img)
            else:
                print(f"Failed to load image: {img_path}")
    return images

### Main functions

In [ ]:
def generate_submission_qst2(query_dir, bbdd_dir):
    
    # LOAD DATA
    # =======================================================
    
    # Load bbdd images
    bbdd_images = load_images_from_directory(bbdd_dir)
    
    # Load query images
    query_images = load_images_from_directory(query_dir)
        
    
    # DETECT PAINTINGS IN QUERIES
    # ========================================================
    
    detected_paintings_folder = "data/detected_paintings"
    
    # Painting detection code/function HERE
    
    # All images are saved in the same temporary directory, in order (/detected_paintings)
    # A list is saved indicating the number of paintings per image (paintings_per_image.pkl)
    
    # Load list containing paintings per image
    with open('paintings_per_image.pkl', 'rb') as file:
        paintings_per_image = pickle.load(file)
    
    
    # REMOVE NOISE FROM QUERY IMAGES
    # =========================================================
    
    denoised_paintings_folder = 'data/denoised_paintings'
    
    # Using denoising method 5
    create_denoised_dataset(
        noisy_dataset_path = detected_paintings_folder,
        denoised_dataset_path = denoised_paintings_folder,
        method='gaussian',
        lowpass_params={'ksize': 3},
        highpass=True,
        noise_method='laplacian',
        noise_threshold=35.35
    )
    
    
    # EXTRACT TEXTURE FEATURES FROM DENOISED QUERIES AND BBDD
    # =========================================================
    
    # Using DCT with best parameters
    color = "V"
    distance_measure = "Cosine"
    block_size = 64
    num_coefs = 64

    # Load denoised query paintings
    query_images = load_and_preprocess_images(denoised_paintings_folder, extension=".jpg")

    # Transform color space of the images
    query_images_color = transform_images_color_space(query_images, color_space=color)
    bbdd_images_color = transform_images_color_space(bbdd_images, color_space=color)

    # Compute the DCT of the images
    query_dct_blocks = compute_images_block_dct(query_images_color, block_size)
    bbdd_dct_blocks = compute_images_block_dct(bbdd_images_color, block_size)

    # Extract first K coefficients of images DCTs
    query_feature_vectors = extract_dct_coefficients_zigzag(query_dct_blocks, num_coefs, block_size)
    bbdd_feature_vectors = extract_dct_coefficients_zigzag(bbdd_dct_blocks, num_coefs, block_size)
    
    
    # CALCULATE DISTANCES AND ORDER RESULTS
    # =========================================================
    
    # Calculate distance matrix
    distance_matrix = create_distance_matrix_vectors(query_feature_vectors, 
                                                     bbdd_feature_vectors,
                                                     distance_measure)
    # Generate sorted results
    results = generate_results(distance_matrix, distance_measure)
    
    
    # GENERATE SUBMISSION
    # =========================================================
    
    # Top K best results
    k=10
    
    # Extracting the top K best results from each prediction
    results_topK = np.array(results)[:,:k].tolist()
    
    # Final sumbission list of lists of lists
    submission = []
    
    i = 0
    for num_paintings in paintings_per_image:
        if num_paintings == 1:
            submission.append([results_topK[i]])
            i += 1
        elif num_paintings == 2:
            submission.append([results_topK[i], results_topK[i+1]])
            i += 2

In [9]:
a = 0
a += 2
a

2

In [12]:
a = [1]
a.append([1])

In [13]:
a

[1, [1]]

In [9]:
with open('result.pkl', 'rb') as file:
    data = pickle.load(file)

In [37]:
data[29]

[[186, 236, 1, 16, 161, 65, 194, 97, 250, 278],
 [1, 194, 186, 97, 250, 201, 184, 165, 182, 65]]

In [ ]:
# 2 al revés
# 5 bad
# 6 al revés
# 7 al revés
# 11 al revés

In [3]:
with open('./data/paintings_per_image.pkl', 'rb') as file:
    data = pickle.load(file)

In [8]:
data

[1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2]